In [ ]:
import numpy as np
import tensorflow as tf
from evaluation import get_mean_cov_for_each_label, frechet_distance
import pandas as pd

# MNIST Evaluation

In [ ]:
# Get real disbrituion for each label.
# Image data: values are transformed to [0, 1],
#             same as that of generator outputs.
# Dictionary stats_real, example: stats[0] = [mean_0, cov_0]
stats_real = get_mean_cov_for_each_label(datasets='mnist')


# Calculate FID for each label.
# 1. For each label i,
#    use 1000 fake images to calculate the generated distribution.
# 2. Then calculate fid for each label using fake & real distribution.
nClass = 10
fake_num = 6400
stats_fake = {}

trained_generator_mnist = model_mnist.g # mnist generator
latent_points, _ = generate_latent_points(model_mnist.z_dim, fake_num, 'uniform')

for i in range(nClass):
    labels = np.full(shape=(fake_num,), fill_value=i, dtype=np.int32)
    labels = tf.one_hot(labels, 10).numpy()
    # Get input to the generator
    batch_z_to_disply = np.concatenate([latent_points, labels], 1)
    X = trained_generator_mnist(batch_z_to_disply, training=False)
    fake_values = X.numpy().reshape(X.shape[0], -1)
    # generated values should be in [0, 1], because used sigmoid activation
    assert np.min(fake_values) >= -0.01 and np.max(fake_values) <= 1.01
    mean = fake_values.mean(axis=0)
    cov = np.cov(fake_values.T) # 784 x 784
    stats_fake[i] = [mean, cov]
    
fid_list = [] # smaller == better
for i in range(nClass):
    m, c = stats_real[i]
    m_fake, c_fake = stats_fake[i]
    fid_list.append(frechet_distance(m, c, m_fake, c_fake))

In [ ]:
# fid_list = [1,1,2,1.0,2,1,0.1,0.01,1,0]
mnist_result = pd.DataFrame()
mnist_result['label'] = [str(i) for i in range(nClass)] + ['average']
mnist_result['Frechet Inception Distance - MNIST'] = list(fid_list) + [np.mean(fid_list)]
mnist_result.set_index('label')

# Fashion-MNIST Evaluation

In [ ]:
# Get real disbrituion for each label.
# Image data: values are transformed to [0, 1],
#             same as that of generator outputs.
# Dictionary stats_real, example: stats[0] = [mean_0, cov_0]
stats_real = get_mean_cov_for_each_label(datasets='fashion_mnist')


# Calculate FID for each label.
# 1. For each label i,
#    use 1000 fake images to calculate the generated distribution.
# 2. Then calculate fid for each label using fake & real distribution.
nClass = 10
fake_num = 6400
stats_fake = {}

trained_generator_fmnist = model_fmnist.g # fmnist generator
latent_points, _ = generate_latent_points(model_fmnist.z_dim, fake_num, 'uniform')

for i in range(nClass):
    labels = np.full(shape=(fake_num,), fill_value=i, dtype=np.int32)
    labels = tf.one_hot(labels, 10).numpy()
    # Get input to the generator
    batch_z_to_disply = np.concatenate([latent_points, labels], 1)
    X = trained_generator_fmnist(batch_z_to_disply, training=False)
    fake_values = X.numpy().reshape(X.shape[0], -1)
    # generated values should be in [0, 1], because used sigmoid activation
    assert np.min(fake_values) >= -0.01 and np.max(fake_values) <= 1.01
    mean = fake_values.mean(axis=0)
    cov = np.cov(fake_values.T) # 784 x 784
    stats_fake[i] = [mean, cov]
    
fid_list = [] # smaller == better
for i in range(nClass):
    m, c = stats_real[i]
    m_fake, c_fake = stats_fake[i]
    fid_list.append(frechet_distance(m, c, m_fake, c_fake))

In [ ]:
fmnist_result = pd.DataFrame()
fmnist_result['label'] = [str(i) for i in range(nClass)] + ['average']
fmnist_result['Frechet Inception Distance - Fashion-MNIST'] = list(fid_list) + [np.mean(fid_list)]
fmnist_result.set_index('label')